In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os
from datetime import datetime  # Import datetime for file naming

# Function to fetch options data
def fetch_options_data(stock_symbol):
    session = requests.Session()
    
    url = f'https://www.nseindia.com/api/option-chain-indices?symbol={stock_symbol}'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': 'https://www.nseindia.com/',
        'X-Requested-With': 'XMLHttpRequest',
    }

    response = session.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        
        if 'records' in data and 'data' in data['records']:
            if len(data['records']['data']) > 0:
                calls_data = []
                puts_data = []

                for option in data['records']['data']:
                    strike_price = option['strikePrice']
                    expiry_date = option['expiryDate']
                    
                    if 'CE' in option:
                        calls_data.append({
                            'strikePrice': strike_price,
                            'expiryDate': expiry_date,
                            **option['CE']
                        })
                    
                    if 'PE' in option:
                        puts_data.append({
                            'strikePrice': strike_price,
                            'expiryDate': expiry_date,
                            **option['PE']
                        })
                
                calls_df = pd.DataFrame(calls_data) if calls_data else pd.DataFrame()
                puts_df = pd.DataFrame(puts_data) if puts_data else pd.DataFrame()

                return calls_df, puts_df
            else:
                print("No options data available for the specified symbol.")
                return None, None
        else:
            print("No 'records' or 'data' found in response.")
            return None, None
    else:
        print(f"Failed to retrieve data: {response.status_code} - {response.text}")
        return None, None

# Load baseline skew from CSV if it exists
def load_baseline_skew(filename):
    if os.path.exists(filename):
        return pd.read_csv(filename)
    return None

# Save baseline skew to CSV
def save_baseline_skew(baseline_skew, filename):
    baseline_skew.to_csv(filename, index=False)

# Function to calculate implied volatility skew with filtering out zero IVs
def calculate_volatility_skew(options_df):
    # Filter out rows where impliedVolatility is 0
    options_df_filtered = options_df[options_df['impliedVolatility'] > 0]
    return options_df_filtered[['strikePrice', 'expiryDate', 'impliedVolatility']]

# Function to plot volatility skew
def plot_volatility_skew(df, option_type):
    plt.figure(figsize=(10, 6))
    
    # Sort expiry dates from nearest to farthest
    sorted_expiry_dates = sorted(df['expiryDate'].unique(), key=pd.to_datetime)
    
    for expiry in sorted_expiry_dates:
        subset = df[df['expiryDate'] == expiry]
        plt.plot(subset['strikePrice'], subset['impliedVolatility'], label=f'Expiry: {expiry}')
    
    plt.title(f'Volatility Skew for {option_type}')
    plt.xlabel('Strike Price')
    plt.ylabel('Implied Volatility')
    plt.legend()
    plt.grid()
    plt.show()

# Function to compare current and baseline skews
def compare_skews(baseline_skew, current_skew, threshold):
    significant_changes = []
    
    # Create dictionaries for easy access
    baseline_dict = {(row['strikePrice'], row['expiryDate']): row['impliedVolatility'] for _, row in baseline_skew.iterrows()}
    current_dict = {(row['strikePrice'], row['expiryDate']): row['impliedVolatility'] for _, row in current_skew.iterrows()}
    
    for key, current_iv in current_dict.items():
        baseline_iv = baseline_dict.get(key)
        
        if baseline_iv is not None and abs(current_iv - baseline_iv) > threshold:
            significant_changes.append((key[0], key[1], current_iv, baseline_iv))  # (strike, expiry, current IV, baseline IV)

    return significant_changes

# Function to alert significant changes
def alert_significant_changes(changes, option_type):
    for strike, expiry, current, baseline in changes:
        print(f"Alert: Significant change detected for {option_type} strike {strike} with expiry {expiry}. "
              f"Current IV: {current}, Baseline IV: {baseline}")

# Function to save significant changes to a file
def save_significant_changes_to_file(significant_changes, option_type):
    date_str = datetime.now().strftime('%Y-%m-%d')
    filename = f'significant_changes_{option_type}_{date_str}.csv'
    
    # Format data for saving
    changes_df = pd.DataFrame(significant_changes, columns=['Strike', 'Expiry', 'Current IV', 'Baseline IV'])
    changes_df.to_csv(filename, index=False)
    print(f"Significant changes saved to {filename}")

# Main execution logic
if __name__ == "__main__":
    stock_symbol = "NIFTY"
    baseline_calls_filename = "baseline_calls_skew.csv"
    baseline_puts_filename = "baseline_puts_skew.csv"

    # Load existing baseline skews for calls and puts
    baseline_calls = load_baseline_skew(baseline_calls_filename)
    baseline_puts = load_baseline_skew(baseline_puts_filename)
    
    # Fetch options data
    calls, puts = fetch_options_data(stock_symbol)

    if calls is not None and puts is not None:
        # Calculate skew for calls and puts, filtering out zero IVs
        calls_data = calculate_volatility_skew(calls)
        puts_data = calculate_volatility_skew(puts)

        # Plot skews
        plot_volatility_skew(calls_data, 'Calls')
        plot_volatility_skew(puts_data, 'Puts')

        # Compare skews if baseline exists
        if baseline_calls is not None and baseline_puts is not None:
            # Set a threshold for significant changes
            threshold = 0.05  # Example: 5%

            # Compare skews
            significant_call_changes = compare_skews(baseline_calls, calls_data, threshold)
            significant_put_changes = compare_skews(baseline_puts, puts_data, threshold)

            # Alert for significant changes
            alert_significant_changes(significant_call_changes, 'Calls')
            alert_significant_changes(significant_put_changes, 'Puts')

            # Save significant changes to files
            if significant_call_changes:
                save_significant_changes_to_file(significant_call_changes, 'Calls')
            if significant_put_changes:
                save_significant_changes_to_file(significant_put_changes, 'Puts')

        # Save the current skews as the new baseline
        save_baseline_skew(calls_data, baseline_calls_filename)
        save_baseline_skew(puts_data, baseline_puts_filename)


Failed to retrieve data: 401 - <!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <title>Resource not found</title>
</head>
<body style="background-color: #e8e8e8;">
  <div style="font-size: 32px; padding: 50px 25px; text-align: center;">
    Resource not found
  </div>
</body>
</html>

